In [19]:
import numpy as np
import polars as pl
import os
import pandas as pd
from tqdm import tqdm
from transformers import AutoModel,AutoTokenizer
import torch
import torch.nn.functional as F
from tqdm import tqdm
import gc
from time import time
from pandarallel import pandarallel

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostClassifier, cv, Pool, CatBoostRegressor
from sklearn.preprocessing import StandardScaler

from annoy import AnnoyIndex
import random
import gc
tqdm.pandas()
pd.set_option('display.max_rows', 1000)


import warnings
warnings.filterwarnings("ignore")

In [61]:
train=pl.read_csv('/kaggle/input/amc-2023/dataset/train.csv').to_pandas()
test=pl.read_csv('/kaggle/input/amc-2023/dataset/test.csv').to_pandas()

train['TITLE']=train['TITLE'].fillna('')
test['TITLE']=test['TITLE'].fillna('')

train['DESCRIPTION']=train['DESCRIPTION'].fillna('')
test['DESCRIPTION']=test['DESCRIPTION'].fillna('')

train['BULLET_POINTS']=train['BULLET_POINTS'].fillna('')
test['BULLET_POINTS']=test['BULLET_POINTS'].fillna('')

test['PRODUCT_LENGTH']=np.nan

In [62]:
import re
def extract_words(text):
    words = re.findall(r'\b\w{2,}\b', text)
    return ' '.join(words[:3])

train['BRAND']=train['TITLE'].progress_apply(extract_words)
test['BRAND']=test['TITLE'].progress_apply(extract_words)



100%|██████████| 734736/734736 [00:05<00:00, 128579.57it/s]


In [104]:
train[train.BULLET_POINTS!='']

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,BRAND,BRAND_PRODUCT_TYPE,brand_prod_enc,brand_enc,...,prod_max_length,brand_mean_length,brand_median_length,brand_max_length,title_mean_length,title_median_length,title_max_length,brand_prod_mean_length,brand_prod_median_length,brand_prod_max_length
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,,1650,2125.980000,ArtzFolio Tulip Flowers,ArtzFolio Tulip Flowers1650,168495,148588,...,292800.000000,1666.401667,2125.980000,2401.570000,2125.980000,2125.980000,2125.980000,2125.980000,2125.980000,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",,2755,393.700000,Marks Spencer Girls,Marks Spencer Girls2755,1174064,1033764,...,4133.858264,393.700000,393.700000,393.700000,393.700000,393.700000,393.700000,393.700000,393.700000,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,PRIKNIK Horn Red,PRIKNIK Horn Red7537,1380680,1212727,...,283897.637506,748.031495,748.031495,748.031495,748.031495,748.031495,748.031495,748.031495,748.031495,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,ALISHAH Women Cotton,ALISHAH Women Cotton2996,58753,54160,...,11456.692902,787.401574,787.401574,787.401574,787.401574,787.401574,787.401574,787.401574,787.401574,787.401574
5,2152929,HINS Metal Bucket Shape Plant Pot for Indoor &...,"[Simple and elegant, great for displaying indo...",HINS Brings you the most Elegant Looking Pot w...,5725,950.000000,HINS Metal Bucket,HINS Metal Bucket5725,813790,719389,...,118110.236100,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000,950.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2249692,2690647,Olmeo 14 LED Round Fog Light 4 Inches Waterpro...,"[Easy to install;,Round shape 14LED spot worki...",,7978,590.550000,Olmeo 14 LED,Olmeo 14 LED7978,1344676,1181156,...,389055.117713,590.550000,590.550000,590.550000,590.550000,590.550000,590.550000,590.550000,590.550000,590.550000
2249693,2422167,Nike Women's As W Ny Df Swsh Hn Kh Bra (CZ7610...,Material : Polyester,,3009,1181.100000,Nike Women As,Nike Women As3009,1307379,1148306,...,10787.380000,1939.858750,1653.540000,3602.355000,1181.100000,1181.100000,1181.100000,1181.100000,1181.100000,1181.100000
2249694,2766635,"(3PCS) Goose Game Cute Cartoon Enamel Pins, Fu...",[❤ [Inspiration] Inspired by the Untitled Goos...,<p><b>[Brand]: </b>XVIEONR</p> <p><br></p> <p>...,3413,125.984252,3PCS Goose Game,3PCS Goose Game3413,29054,26533,...,74803.000000,125.984252,125.984252,125.984252,125.984252,125.984252,125.984252,125.984252,125.984252,125.984252
2249695,1987786,Kangroo Sweep Movement Printed Wooden Wall Clo...,"[Dial size: 12 inches in diameter,Big, clear r...",Wall Clocks Are Very Attractive In Looks And E...,1574,1200.000000,Kangroo Sweep Movement,Kangroo Sweep Movement1574,992559,873908,...,10236.200000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000


In [64]:
# train['BRAND_BULLET']=train['BULLET_POINTS'].progress_apply(extract_words)
# test['BRAND_BULLET']=test['BULLET_POINTS'].progress_apply(extract_words)

In [65]:
# train['BRAND_DESC']=train['BULLET_POINTS'].progress_apply(extract_words)
# test['BRAND_BULLET']=test['BULLET_POINTS'].progress_apply(extract_words)

In [67]:
train['BRAND_PRODUCT_TYPE']=train['BRAND']+train['PRODUCT_TYPE_ID'].astype(str)
test['BRAND_PRODUCT_TYPE']=test['BRAND']+test['PRODUCT_TYPE_ID'].astype(str)

In [68]:
def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

In [70]:
from sklearn.preprocessing import LabelEncoder

train_df=pd.concat([train,test],axis=0)

le=LabelEncoder()
train_df['brand_prod_enc']=le.fit_transform(train_df['BRAND_PRODUCT_TYPE'])

le=LabelEncoder()
train_df['brand_enc']=le.fit_transform(train_df['BRAND'])

le=LabelEncoder()
train_df['title_enc']=le.fit_transform(train_df['TITLE'])

le=LabelEncoder()
train_df['bull_enc']=le.fit_transform(train_df['BULLET_POINTS'])

In [72]:
train=train_df.iloc[:len(train)]
test=train_df.iloc[len(train):]

In [116]:
train['prod_mean_length'] =train['PRODUCT_TYPE_ID'].map(dict(train.groupby('PRODUCT_TYPE_ID').PRODUCT_LENGTH.mean()))
train['prod_median_length'] =train['PRODUCT_TYPE_ID'].map(dict(train.groupby('PRODUCT_TYPE_ID').PRODUCT_LENGTH.median()))
train['prod_max_length'] =train['PRODUCT_TYPE_ID'].map(dict(train.groupby('PRODUCT_TYPE_ID').PRODUCT_LENGTH.max()))
train['prod_min_length'] =train['PRODUCT_TYPE_ID'].map(dict(train.groupby('PRODUCT_TYPE_ID').PRODUCT_LENGTH.min()))

# train['brand_mean_length'] =train['BRAND'].map(dict(train.groupby('BRAND').PRODUCT_LENGTH.mean()))
# train['brand_median_length'] =train['BRAND'].map(dict(train.groupby('BRAND').PRODUCT_LENGTH.median()))
# train['brand_max_length'] =train['BRAND'].map(dict(train.groupby('BRAND').PRODUCT_LENGTH.max()))
# train['brand_min_length'] =train['BRAND'].map(dict(train.groupby('BRAND').PRODUCT_LENGTH.min()))

# train['title_mean_length'] =train['TITLE'].map(dict(train.groupby('TITLE').PRODUCT_LENGTH.mean()))
# train['title_median_length'] =train['TITLE'].map(dict(train.groupby('TITLE').PRODUCT_LENGTH.median()))
# train['title_max_length'] =train['TITLE'].map(dict(train.groupby('TITLE').PRODUCT_LENGTH.max()))
# train['title_min_length'] =train['TITLE'].map(dict(train.groupby('TITLE').PRODUCT_LENGTH.min()))

# train['brand_prod_mean_length'] =train['BRAND_PRODUCT_TYPE'].map(dict(train.groupby('BRAND_PRODUCT_TYPE').PRODUCT_LENGTH.mean()))
# train['brand_prod_median_length'] =train['BRAND_PRODUCT_TYPE'].map(dict(train.groupby('BRAND_PRODUCT_TYPE').PRODUCT_LENGTH.median()))
# train['brand_prod_max_length'] =train['BRAND_PRODUCT_TYPE'].map(dict(train.groupby('BRAND_PRODUCT_TYPE').PRODUCT_LENGTH.max()))
# train['brand_prod_min_length'] =train['BRAND_PRODUCT_TYPE'].map(dict(train.groupby('BRAND_PRODUCT_TYPE').PRODUCT_LENGTH.min()))


In [107]:
model = CatBoostRegressor(iterations = 100, depth = 16, cat_features=[0, 1, 2, 3, 4], eval_metric='MAPE', loss_function='MAPE')

In [117]:
features = ['brand_prod_enc', 'brand_enc', 'title_enc', 'bull_enc', 'PRODUCT_TYPE_ID', \
          'prod_mean_length', 'prod_median_length', 'prod_max_length', 'prod_min_length']
#            'brand_mean_length', 'brand_median_length', 'brand_max_length', \
#            'title_mean_length', 'title_median_length', 'title_max_length', \
#            'brand_prod_mean_length', 'brand_prod_median_length', 'brand_prod_max_length']
model.fit(train[features], train['PRODUCT_LENGTH'])

0:	learn: 0.9662319	total: 3.73s	remaining: 6m 9s
1:	learn: 0.9614526	total: 6.77s	remaining: 5m 31s
2:	learn: 0.9586333	total: 9.78s	remaining: 5m 16s
3:	learn: 0.9546042	total: 13.1s	remaining: 5m 14s
4:	learn: 0.9505629	total: 16.1s	remaining: 5m 5s
5:	learn: 0.9470744	total: 19.1s	remaining: 4m 58s
6:	learn: 0.9429221	total: 21.9s	remaining: 4m 51s
7:	learn: 0.9391036	total: 25.3s	remaining: 4m 50s
8:	learn: 0.9345990	total: 28.4s	remaining: 4m 46s
9:	learn: 0.9301037	total: 31.5s	remaining: 4m 43s
10:	learn: 0.9258476	total: 34.5s	remaining: 4m 38s
11:	learn: 0.9210781	total: 37.5s	remaining: 4m 34s
12:	learn: 0.9162854	total: 40.4s	remaining: 4m 30s
13:	learn: 0.9124266	total: 43.3s	remaining: 4m 26s
14:	learn: 0.9085467	total: 46.3s	remaining: 4m 22s
15:	learn: 0.9041218	total: 49.4s	remaining: 4m 19s
16:	learn: 0.8999610	total: 52.4s	remaining: 4m 15s
17:	learn: 0.8958681	total: 55.4s	remaining: 4m 12s
18:	learn: 0.8916880	total: 58.4s	remaining: 4m 8s
19:	learn: 0.8881857	tota

In [118]:
preds = model.predict(train[features])

In [119]:
comp_score(train['PRODUCT_LENGTH'], preds)

(0.6844911727482089, 31.550882725179108)

In [120]:
test['prod_mean_length'] =test['PRODUCT_TYPE_ID'].map(dict(train.groupby('PRODUCT_TYPE_ID').PRODUCT_LENGTH.mean()))
test['prod_median_length'] =test['PRODUCT_TYPE_ID'].map(dict(train.groupby('PRODUCT_TYPE_ID').PRODUCT_LENGTH.median()))
test['prod_max_length'] =test['PRODUCT_TYPE_ID'].map(dict(train.groupby('PRODUCT_TYPE_ID').PRODUCT_LENGTH.max()))
test['prod_min_length'] =test['PRODUCT_TYPE_ID'].map(dict(train.groupby('PRODUCT_TYPE_ID').PRODUCT_LENGTH.min()))

# test['brand_mean_length'] =test['BRAND'].map(dict(train.groupby('BRAND').PRODUCT_LENGTH.mean()))
# test['brand_median_length'] =test['BRAND'].map(dict(train.groupby('BRAND').PRODUCT_LENGTH.median()))
# test['brand_max_length'] =test['BRAND'].map(dict(train.groupby('BRAND').PRODUCT_LENGTH.max()))

# test['title_mean_length'] =test['TITLE'].map(dict(train.groupby('TITLE').PRODUCT_LENGTH.mean()))
# test['title_median_length'] =test['TITLE'].map(dict(train.groupby('TITLE').PRODUCT_LENGTH.median()))
# test['title_max_length'] =test['TITLE'].map(dict(train.groupby('TITLE').PRODUCT_LENGTH.max()))

# test['brand_prod_mean_length'] =test['BRAND_PRODUCT_TYPE'].map(dict(train.groupby('BRAND_PRODUCT_TYPE').PRODUCT_LENGTH.mean()))
# test['brand_prod_median_length'] =test['BRAND_PRODUCT_TYPE'].map(dict(train.groupby('BRAND_PRODUCT_TYPE').PRODUCT_LENGTH.median()))
# test['brand_prod_max_length'] =test['BRAND_PRODUCT_TYPE'].map(dict(train.groupby('BRAND_PRODUCT_TYPE').PRODUCT_LENGTH.max()))


In [91]:
null_fe = ['prod_mean_length', 'prod_median_length', 'prod_max_length', \
           'brand_mean_length', 'brand_median_length', 'brand_max_length', \
           'title_mean_length', 'title_median_length', 'title_max_length', \
           'brand_prod_mean_length', 'brand_prod_median_length', 'brand_prod_max_length']

for fe in null_fe:
    test[fe].fillna(np.mean(train[fe]), inplace=True)

In [132]:
test['PRODUCT_LENGTH']=model.predict(test[features])

In [123]:
label_enc=pd.DataFrame(train['brand_prod_enc'].value_counts())
label_enc[label_enc['brand_prod_enc']>6].index

Int64Index([ 119500, 2006235, 1932147,  746574, 1497518, 1932154, 1749078,
            1736792,  747037,  730554,
            ...
            1948490,  750371, 1715945, 1718586, 1808977, 1542694, 1003841,
             534989, 1226837, 1346047],
           dtype='int64', length=17673)

In [126]:
def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))

# test['preds']=np.nan
preds=np.zeros(train.shape[0])
anomaly=[]

train['preds']=np.nan
for i in tqdm(label_enc[label_enc['brand_prod_enc']>5].index):
    temp=train[train['brand_prod_enc']==i]['PRODUCT_LENGTH'].values
    ind=train[train['brand_prod_enc']==i]['PRODUCT_LENGTH'].index


    val2=getVal(temp)
    mape,score2=comp_score(temp,[val2]*len(temp))
    preds[ind]=val2
    val=val2
    score=score2
        
        
    train.loc[train['brand_prod_enc']==i,'PRODUCT_LENGTH']=min(val,5000)
    test.loc[test['brand_prod_enc']==i,'PRODUCT_LENGTH']=min(val,5000)

100%|██████████| 22400/22400 [11:38<00:00, 32.06it/s]


In [128]:
comp_score(train[train['PRODUCT_LENGTH'].notna()]['PRODUCT_LENGTH'].values,train[train['PRODUCT_LENGTH'].notna()]['PRODUCT_LENGTH'])

(0.0, 100.0)

In [ ]:
train['preds'].value_counts()

In [ ]:
test.preds.value_counts()

In [2]:
def getVal(dist):
    median = np.power(np.mean(np.power(dist, -1)), -1)
    return median

def comp_score(y_true,y_pred):
    score=0
    score = mean_absolute_percentage_error(y_true,y_pred)                                      
    return score,max(0,100*(1-score))


preds=np.zeros(train.shape[0])
anomaly=[]

for i in tqdm(train['PRODUCT_TYPE_ID'].unique()):
    temp=train[(train['PRODUCT_TYPE_ID']==i)&(train['preds'].isna())]['PRODUCT_LENGTH'].values
    ind=train[(train['PRODUCT_TYPE_ID']==i)&(test['preds'].isna())]['PRODUCT_LENGTH'].index

#     val=np.median(temp)
#     preds[ind]=val
#     mape,score=comp_score(temp,[val]*len(temp))
    
    
#     val2=np.min(temp)*2
#     mape,score2=comp_score(temp,[val2]*len(temp))
    
#     if(score2>score):
#         preds[ind]=val2
#         val=val2
#         score=score2
 

#     val2=np.mean(temp)
#     mape,score2=comp_score(temp,[val2]*len(temp))
    
#     if(score2>score):
#         preds[ind]=val2
#         val=val2
#         score=score2
 
    val2=getVal(temp)
    mape,score2=comp_score(temp,[val2]*len(temp))
    preds[ind]=val2
    val=val2
    score=score2
        
    train.loc[(train['PRODUCT_TYPE_ID']==i)&(train['preds'].isna()),'preds']=min(val,5000)
    test.loc[(test['PRODUCT_TYPE_ID']==i)&(test['preds'].isna()),'preds']=min(val,5000)

NameError: name 'train' is not defined

In [3]:
hm=getVal(train['PRODUCT_LENGTH'])

train['preds'].fillna(hm,inplace=True)
test['preds'].fillna(hm,inplace=True)

NameError: name 'train' is not defined

In [ ]:
comp_score(train['PRODUCT_LENGTH'].values,train['preds'].values)

In [133]:
# test['PRODUCT_LENGTH']=test['preds']
submission=pd.read_csv('/kaggle/input/amc-2023/dataset/sample_submission.csv')
submission.rename(columns={'PRODUCT_LENGTH':'L'},inplace=True)

In [134]:
submission=submission.merge(test[['PRODUCT_ID','PRODUCT_LENGTH']],on='PRODUCT_ID')

In [135]:
submission[['PRODUCT_ID','PRODUCT_LENGTH']].to_csv('submission.csv',index=False)

In [96]:
submission

,PRODUCT_ID,L,PRODUCT_LENGTH
0,604373,701.093794,671.792872
1,1729783,734.506163,838.601117
2,1871949,741.360258,624.955890
3,1107571,730.327767,549.770540
4,624253,666.847946,642.448887
...,...,...,...
734731,921419,733.838809,515.338788
734732,2456362,746.810825,507.979836
734733,841529,691.127128,591.218783
734734,1190194,757.643591,723.858571


In [ ]:
train.head()

In [ ]:
test['preds'].value_counts()

In [ ]:
train['preds'].value_counts()